In [ ]:
%matplotlib inline


Generate target values
======================


In this example,
we generate **target values** for a problem
based on the performances of an algorithm configuration.

Imports
-------
We start by making the necessary imports.



In [ ]:
from __future__ import annotations

from gemseo import compute_doe
from gemseo import configure
from gemseo.problems.optimization.power_2 import Power2

from gemseo_benchmark.algorithms.algorithm_configuration import AlgorithmConfiguration
from gemseo_benchmark.algorithms.algorithms_configurations import (
    AlgorithmsConfigurations,
)
from gemseo_benchmark.problems.problem import Problem

Let us consider the problem [Power2][gemseo.problems.analytical.power_2.Power2]
already implemented in GEMSEO.



In [ ]:
problem = Problem(
    name="Power2",
    optimization_problem_creator=Power2,
    optimum=Power2.get_solution()[1],
)

We define ten starting points by optimized Latin hypercube sampling (LHS).



In [ ]:
design_space = problem.creator().design_space
problem.start_points = compute_doe(design_space, "OT_OPT_LHS", 10)

Let use the optimizer COBYLA to generate performance histories on the problem.



In [ ]:
algorithms_configurations = AlgorithmsConfigurations(
    AlgorithmConfiguration(
        "NLOPT_COBYLA",
        max_iter=65,
        eq_tolerance=1e-4,
        ineq_tolerance=0.0,
    )
)

Here we choose to deactivate the functions counters, progress bars and bounds check
of GEMSEO to accelerate the script.



In [ ]:
configure(
    enable_function_statistics=False,
    enable_progress_bar=False,
    check_desvars_bounds=False,
)

Let us compute five target values for the problem.
This automatic procedure has two stages:

1. execution of the specified algorithms once for each of the starting points,
2. automatic selection of target values based on the algorithms histories.

These targets represent the milestones of the problem resolution.



In [ ]:
problem.compute_targets(5, algorithms_configurations, best_target_tolerance=1e-5)

We can plot the algorithms histories used as reference
for the computation of the target values,
with the objective value on the vertical axis
and the number of functions evaluations on the horizontal axis.



In [ ]:
problem.targets_generator.plot_histories(problem.optimum, show=True)

Finally, we can plot the target values:
the objective value of each of the five targets is represented
on the vertical axis with a marker indicating whether the target is feasible or not.



In [ ]:
problem.target_values.plot()